In [66]:
import pandas as pd 
import pathlib 
import os 
import warnings
warnings.filterwarnings("ignore")

raw_data_dir = "/media/hieunguyen/HD01/kinase_prediction/raw_data"

# data_type = "proteomics"
data_type = "bulkRNA"

manifestdir = "./manifests"

save_missing_dir = os.path.join(manifestdir, f"{data_type}_missing_files")
os.system(f"mkdir -p {save_missing_dir}")

if data_type == "proteomics":
    all_downloaded_files = [item for item in pathlib.Path(os.path.join(raw_data_dir, data_type)).rglob("**/*mzML.gz")]
    all_manifests = [item for item in pathlib.Path(os.path.join(manifestdir, data_type)).glob("*/*.tsv")]
    manifestdf = pd.DataFrame()
    for inputfile in all_manifests:
        tmpdf = pd.read_csv(inputfile, sep = "\t")
        manifestdf = pd.concat([manifestdf, tmpdf], ignore_index=True)
    filename_col = "File Name"
else:
    all_downloaded_files = [item for item in pathlib.Path(os.path.join(raw_data_dir, data_type)).rglob("**/*_star_gene_counts.tsv")]
    all_manifests = [item for item in pathlib.Path(os.path.join(manifestdir, data_type)).glob("star_gene_counts_only_*.txt")]
    manifestdf = pd.DataFrame()
    for inputfile in all_manifests:
        tmpdf = pd.read_csv(inputfile, sep = "\t")
        manifestdf = pd.concat([manifestdf, tmpdf], ignore_index=True)
    filename_col = "filename"
downloaddf = pd.DataFrame(data = [item.name for item in all_downloaded_files], columns = ["filename"])
downloaddf["path"] = [str(item) for item in all_downloaded_files]

print("List of missing file: ")
missing_files = [item for item in manifestdf[filename_col].unique() if item not in downloaddf["filename"].values]
print(missing_files)

print("saving new manifest file for missing data ...")
manifestdf[manifestdf[filename_col].isin(missing_files)].to_csv(os.path.join(save_missing_dir, f"{data_type}_missing_files.tsv"), sep="\t", index=False)

downloaddf = downloaddf.merge(manifestdf, right_on = filename_col, left_on = "filename")
downloaddf["dataset"] = downloaddf["path"].apply(lambda x: [item for item in x.split("/") if "star_gene_counts" in str(item) and ".tsv" not in item][0])

List of missing file: 
[]
saving new manifest file for missing data ...


In [68]:
downloaddf.groupby("dataset")["filename"].count()

dataset
star_gene_counts_only_bulkRNA_phs000892     340
star_gene_counts_only_bulkRNA_phs001287    2393
Name: filename, dtype: int64

In [73]:
downloaddf[downloaddf["id"] == "45b99ac4-0859-4a3d-8b1e-ce5590_D2"]

,filename,path,id,md5,size,state,dataset
